## Phase - 2: Predicting Anomalies in Network Traffic (A Binary Classification problem)

Before working on this phase, please practice "Activities 4, 5, and 6" in the 'Neural networks using Tensorflow' crash course (see nn-tf tab). 

The main goal in this phase is to experiment and find what network size is needed to 'overfit' the entire dataset at your hand.

In other words, we want to determine how big architecture we need to overfit the data. 

     The place to start is to use 'logistic regression' model and train for as many epochs as needed to obtain as high accuracy as possible. After training hundreds of epochs if you observe that the accuracy is not increasing then it implies that the number of neurons in your model (only one) may not be sufficient for overfitting. 
      The next step is to grow your model into a multi-layer model and add a few neurons (say only 2) in the input layer. This way your model will have '2 + 1 = 3' neurons in total. 
       
       If your accuracy still does not each a 100% or close to 100% you can continue to increase the number of layers and number of neurons. Once you have obtained 100% accuracy (or around 100%) your experiments for this phase are complete. The results of this experiment also inform us that our final model (in subsequent phases) should be smaller than this model. Small here refers to number of layers and number of neurons in each layer.

[BONUS] You will be eligible to receive up to 2 bonus points if you complete this phase without using the Keras/Tenforflow library, i.e., if you implement your own Python code to build neural network (or logistic regression model) and code a function that serves as an optimizer (for example, gradient descent algorithm) to train your model.

'protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login'

In [59]:
catg_cols = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']

#### 2.2 Continuous Columns

In [60]:
cont_cols = [ 'duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

#### 2.3 Target Variable

The target variable "label" contains all different types of malware attacks and also the value "normal" (i.e., not an attack) as shown below.

'normal', 'buffer_overflow', 'loadmodule', 'perl', 'neptune', 'smurf', 'guess_passwd', 'pod', 'teardrop', 'portsweep', 'ipsweep', 'land', 'ftp_write', 'back', 'imap', 'satan', 'phf', 'nmap', 'multihop', 'warezmaster', 'warezclient', 'spy', 'rootkit'

All malware attacks are grouped (transformed) to the value "abnormal" to make this problem a binary classification problem instead of a multi-class classification.


In [61]:
target_variable = 'label'

#### 2.4 Data Load

In [62]:
import pandas as pd
df = pd.read_csv("../datasets/kddcup99_csv.csv")

#### 2.5 Dataset size

In [63]:
df.shape

(494020, 42)

494020 records with 41 features and 1 target variable ("label") for prediction

In [64]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


As seen above, there are various type of malware attacks which can be grouped as "abnormal" to make this problem as binary classification problem

##### 2.6.1 Group all malware attacks as "abnormal"

In [66]:
attack_types = list(df['label'].unique())
attack_types.remove('normal') # remove normal from attack types as we only want malware attacks to convert as abnormal

In [67]:
df['label'] = df['label'].replace(attack_types, 'abnormal')
df.shape

(494020, 42)

In [68]:
df['label'].value_counts()

abnormal    396743
normal       97277
Name: label, dtype: int64

In [71]:
df[catg_cols].dtypes

protocol_type     object
service           object
flag              object
land               int64
logged_in          int64
is_host_login      int64
is_guest_login     int64
dtype: object

As seen from the above datatypes of the categorical columns,the column values are not strings. We need to convert them to string before performing any analysis

##### 2.7.1 Fixing the data types of categorical columns

In [72]:
df['protocol_type'] = df['protocol_type'].astype(str)
df['service'] = df['service'].astype(str)
df['flag'] = df['flag'].astype(str)
df['land'] = df['land'].astype(str)
df['logged_in'] = df['logged_in'].astype(str)
df['is_host_login'] = df['is_host_login'].astype(str)
df['is_guest_login'] = df['is_guest_login'].astype(str)
df[catg_cols].dtypes

protocol_type     object
service           object
flag              object
land              object
logged_in         object
is_host_login     object
is_guest_login    object
dtype: object

In [84]:
df.drop(columns=['is_host_login'], inplace=True)
df.drop(columns=['num_outbound_cmds'], inplace=True)

### 3. Transform Target Binary label to 0 and 1

In [85]:
df['label'] = df['label'].map({'normal': 1, 'abnormal': 0})
df['label'].value_counts()

0    396743
1     97277
Name: label, dtype: int64

### 3. One hot encode categorical columns

In [86]:
def convert_categorical_to_one_hot_encoding(dataset, column_name):
    dummy_values = pd.get_dummies(dataset[column_name])
    
    for category in dummy_values.columns:
        dummy_value_name = f"{column_name}-{category}"
        dataset[dummy_value_name] = dummy_values[category]
    dataset.drop(column_name, axis=1, inplace=True)



In [87]:
for cat_col in catg_cols:
    if cat_col != "is_host_login":
        convert_categorical_to_one_hot_encoding(df, cat_col)
df.shape

(494020, 120)

In [88]:
df.head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag-S2,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_guest_login-0,is_guest_login-1
0,0,181,5450,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
1,0,239,486,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
2,0,235,1337,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
3,0,219,1337,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
4,0,217,2032,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0


###### a. Shuffle DataFrame

In [91]:
df = df.sample(frac=1).reset_index(drop=True)

### 3. Data Normalization (scaling numerical features to unit norm)

As seen from the above continous variables distribution in the section 2.6 the scale of some of the features (columns) values are not in same range as others.
Hence the features need to be normalized so that the optimization during model training will happen in a better way and the model will not be sensitive to the features.

In [96]:
cont_cols_new = [each for each in cont_cols if each!='num_outbound_cmds']

In [113]:
df_CONTCOLS_MIN = df[cont_cols_new].min(axis=0)
df_CONTCOLS_MAX = df[cont_cols_new].max(axis=0)

In [114]:
df[cont_cols_new] = (df[cont_cols_new] - df_CONTCOLS_MIN) / (df_CONTCOLS_MAX - df_CONTCOLS_MIN)

In [115]:
df[cont_cols_new].head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.078431,0.08,0.07,0.00,0.00,0.0,0.0,1.0,1.0
1,0.000000,1.488371e-06,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0
2,0.000000,2.855595e-07,0.000125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.121569,1.000000,1.00,0.00,0.03,0.07,0.0,0.0,0.0,0.0
3,0.296593,2.105641e-07,0.000020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.003922,0.00,0.67,0.98,0.00,0.0,0.0,0.0,0.0
4,0.000000,1.488371e-06,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0


In [116]:
df.isna().values.any()

False

In [103]:
x_train_numpy = XTRAIN.to_numpy()
y_train_numpy = YTRAIN.to_numpy()

In [117]:
X_numpy = df.drop(columns=['label']).to_numpy()
Y_numpy = df['label'].to_numpy()

In [119]:
X_numpy.shape

(494020, 119)

In [121]:
Y_numpy.shape

(494020,)

In [122]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(1, input_dim = len(X_numpy[0, :]), activation='sigmoid'))

In [123]:
model.compile(loss='binary_crossentropy', optimizer = 'rmsprop', metrics=['accuracy'])
model.fit(x_train_numpy, y_train_numpy, epochs = 256, verbose = 1)

Epoch 1/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0540 - accuracy: 0.9816
Epoch 2/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0283 - accuracy: 0.9915
Epoch 3/256
10807/10807 [==============================] - 14s 1ms/step - loss: 0.0255 - accuracy: 0.9915
Epoch 4/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0240 - accuracy: 0.9915
Epoch 5/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0230 - accuracy: 0.9915
Epoch 6/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0225 - accuracy: 0.9915
Epoch 7/256
10807/10807 [==============================] - 14s 1ms/step - loss: 0.0219 - accuracy: 0.9915
Epoch 8/256
10807/10807 [==============================] - 13s 1ms/step - loss: 0.0216 - accuracy: 0.9916
Epoch 9/256
10807/10807 [==============================] - 14s 1ms/step - loss: 0.0211 - accuracy: 0.9917
Epoch 10/256
10807/10807 [====================

10807/10807 [==============================] - 16s 2ms/step - loss: 0.0213 - accuracy: 0.9978
Epoch 78/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0214 - accuracy: 0.9978
Epoch 79/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0215 - accuracy: 0.9978
Epoch 80/256
10807/10807 [==============================] - 16s 1ms/step - loss: 0.0216 - accuracy: 0.9978
Epoch 81/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0217 - accuracy: 0.9978
Epoch 82/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0218 - accuracy: 0.9978
Epoch 83/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0219 - accuracy: 0.9978
Epoch 84/256
10807/10807 [==============================] - 16s 1ms/step - loss: 0.0219 - accuracy: 0.9978
Epoch 85/256
10807/10807 [==============================] - 13s 1ms/step - loss: 0.0219 - accuracy: 0.9978
Epoch 86/256
10807/10807 [========================

10807/10807 [==============================] - 15s 1ms/step - loss: 0.0263 - accuracy: 0.9980
Epoch 154/256
10807/10807 [==============================] - 14s 1ms/step - loss: 0.0264 - accuracy: 0.9980
Epoch 155/256
10807/10807 [==============================] - 13s 1ms/step - loss: 0.0264 - accuracy: 0.9980
Epoch 156/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0265 - accuracy: 0.9980
Epoch 157/256
10807/10807 [==============================] - 14s 1ms/step - loss: 0.0265 - accuracy: 0.9980
Epoch 158/256
10807/10807 [==============================] - 14s 1ms/step - loss: 0.0266 - accuracy: 0.9980
Epoch 159/256
10807/10807 [==============================] - ETA: 0s - loss: 0.0267 - accuracy: 0.99 - 14s 1ms/step - loss: 0.0267 - accuracy: 0.9980
Epoch 160/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0268 - accuracy: 0.9980
Epoch 161/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0268 - accuracy: 0.9980


10807/10807 [==============================] - 16s 1ms/step - loss: 0.0309 - accuracy: 0.9980
Epoch 229/256
10807/10807 [==============================] - 16s 1ms/step - loss: 0.0309 - accuracy: 0.9980
Epoch 230/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0310 - accuracy: 0.9980
Epoch 231/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0311 - accuracy: 0.9980
Epoch 232/256
10807/10807 [==============================] - 15s 1ms/step - loss: 0.0311 - accuracy: 0.9980
Epoch 233/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0312 - accuracy: 0.9980
Epoch 234/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0312 - accuracy: 0.9980
Epoch 235/256
10807/10807 [==============================] - 16s 2ms/step - loss: 0.0313 - accuracy: 0.9980
Epoch 236/256
10807/10807 [==============================] - 16s 1ms/step - loss: 0.0314 - accuracy: 0.9980
Epoch 237/256
10807/10807 [===============

In [124]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 120       
Total params: 120
Trainable params: 120
Non-trainable params: 0
_________________________________________________________________


### Model : 2. With more layers and neurons so that dataset can overfit

In [126]:
model_2 = Sequential()
model_2.add(Dense(8, input_dim = len(X_numpy[0, :]), activation='relu'))
model_2.add(Dense(4, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))

In [ ]:
model_2.compile(loss='binary_crossentropy', optimizer = 'rmsprop', metrics=['accuracy'])
model_2.fit(x_train_numpy, y_train_numpy, epochs = 32, verbose = 1)

Epoch 1/32
10807/10807 [==============================] - 6s 483us/step - loss: 0.0882 - accuracy: 0.9919
Epoch 2/32
10807/10807 [==============================] - 5s 479us/step - loss: 0.0197 - accuracy: 0.9968
Epoch 3/32
10807/10807 [==============================] - 5s 481us/step - loss: 0.0184 - accuracy: 0.9973
Epoch 4/32
10807/10807 [==============================] - 5s 486us/step - loss: 0.0184 - accuracy: 0.9975
Epoch 5/32
10807/10807 [==============================] - 5s 482us/step - loss: 0.0183 - accuracy: 0.9975
Epoch 6/32
10807/10807 [==============================] - 5s 495us/step - loss: 0.0181 - accuracy: 0.9976
Epoch 7/32
10807/10807 [==============================] - 5s 486us/step - loss: 0.0180 - accuracy: 0.9976
Epoch 8/32
10807/10807 [==============================] - 5s 480us/step - loss: 0.0179 - accuracy: 0.9976
Epoch 9/32
10807/10807 [==============================] - 5s 493us/step - loss: 0.0178 - accuracy: 0.9977
Epoch 10/32
10807/10807 [=====================

In [110]:
print ('True Validation Data:')
print(y_train_numpy[:10])
prediction = model.predict(x_train_numpy)
print ('Prediction:')
print(prediction[0:10].T)

True Validation Data:
[0 0 0 1 0 1 1 0 0 0]
Prediction:
[[7.0481242e-06 2.2987747e-08 6.7529127e-06 9.7435486e-01 6.7563465e-06
  9.8804760e-01 9.9964744e-01 6.7529127e-06 6.7529127e-06 6.8989330e-06]]
